In [2]:
import pandas as pd
import os
import numpy as np

data_df = pd.read_csv(os.path.join('.','wiki-Vote.txt'),sep='	', header=None, skiprows =3)

data_array =np.array(data_df,dtype=int)

node_list = np.unique(data_array)
print('The number of nodes in the network is %d' % len(node_list))
print('The number of directed edges in the network is  %d' % len(data_array))

The number of nodes in the network is 7115
The number of directed edges in the network is  103689


点评:挺简洁，赞！

In [3]:
data_array

array([[  30, 1412],
       [  30, 3352],
       [  30, 5254],
       ...,
       [8150, 8275],
       [8150, 8276],
       [8274, 8275]])

In [4]:
node_list

array([   3,    4,    5, ..., 8295, 8296, 8297])

In [5]:
out_list = list(np.unique(data_array[:,0]))

undirected_edges = 0
for i in out_list:
    in_tmp_array = data_array[data_array[:,0] ==i,1]
    for j in in_tmp_array:
        out_in_tmp_array = data_array[data_array[:,0] ==j]
        undirected_edges += np.sum(out_in_tmp_array[:,1]==i)
    
print('The number of undirected edges in the network  %d' % int(undirected_edges/2))

The number of undirected edges in the network  2927


**点评:计算双向边的效率真是差，正常的做法时间复杂最多O(n^2),你这个代码为O(n^3),研究了下才明白，之所以绕一次去判断i=j并且j=i，是因为没有按点构建索引导致，**

In [6]:
in_list = list(np.unique(data_array[:,1]))
zero_in_note,zero_out_note = 0,0
for i in in_list:
    if np.sum(data_array[:,0] ==i) == 0:
       zero_out_note +=1
for i in out_list:
    if np.sum(data_array[:,1] ==i) == 0:
       zero_in_note +=1
       
print('The number of nodes of zero out-degree in the network is %d' % zero_out_note)
print('The number of nodes of zero in-degree in the network is %d' % zero_in_note)

The number of nodes of zero out-degree in the network is 1005
The number of nodes of zero in-degree in the network is 4734


**点评:既然用了np.unique的方法，计算入库为零和出度为零可以更简单，total-len(out_list)就是出度为零，total-len(in_list)就是入度为零**

In [11]:
def array_to_adjacency_list(data_array):
    out_list = np.unique(data_array[:,0])
    adjacency_list = dict()
    for i in out_list:
        in_tmp_array = data_array[data_array[:,0] ==i,1]
        set_array = set(in_tmp_array)
        adjacency_list.update({i:set_array})
    return adjacency_list
    
adjacency_list = array_to_adjacency_list(data_array)

In [12]:
def re_tr(G):#翻转图
    GT = {}
    for u in G:
        for v in G[u]:
            # print(GT)
            if GT.get(v):
                GT[v].add(u)
            else:
                GT[v] = set()
                GT[v].add(u)

    return GT
    
def rec_dfs(G,s,S=None):#递归实现深度优先排序
    if S is None:
        #S = set()    #集合存储已经遍历过的节点
        S = list()    #用列表可以更方便查看遍历的次序，而用集合可以方便用difference求差集
    # S.add(s)
    S.append(s)
    #print(S)
    for u in G[s]:
        if u in S:continue
#         if u in G:
        rec_dfs(G,u,S)
    return S

#遍历有向图的强连通分量
def walk(G,start,S=set()):     #传入的参数S，即上面的seen很关键，这避免了通过连通图之间的路径进行遍历
    P,Q = dict(),set()      #list存放遍历顺序，set存放已经遍历过的节点     
    P[start] = None
    Q.add(start)
    while Q:
        u = Q.pop()                      #选择下一个遍历节点（随机性）
        for v in G[u].difference(P,S):         #返回差集
            Q.add(v)
            P[v] = u
    #print(P)    
    return P
    

def scc(G):#获得各个强连通图
    GT = re_tr(G)
    sccs,seen = [],set()
    for u in rec_dfs(G,30):    #以30为起点
        if u in seen:continue
        C = walk(GT,u,seen)
        seen.update(C)
        sccs.append(C)
    return sccs
# scc_data = scc(adjacency_list)
scc_len_list = [len(x) for x in scc_data]
def sum_set(dict_c):
    sum_c = 0
    for x in dict_c:
        sum_c += len(dict_c[x])
        
scc_edge_list = [sum_set(x) for x in scc_data]
                
print('The number of nodes and edges in largest strongly connected component is %d, %d' % \
      (np.max(scc_len_list),np.max(scc_edge_list)))

KeyError: 7